# Knowledge Graph (KG) Reasoner

Based on Animal Farm >> By author: George Orwell (pseudonym of Eric Blair) (1903-1950) http://gutenberg.net.au/ebooks01/0100011.txt


<img src="https://upload.wikimedia.org/wikipedia/commons/f/fb/Animal_Farm_-_1st_edition.jpg" width=300>

# 1. Import Library

In [1]:
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import logging
import spacy
import opennre # https://github.com/thunlp/OpenNRE
from tqdm.notebook import tqdm
import transformers

# 2. Initialize working environment

### Connect to Neo4j

In [2]:
# Initialize the graph DB and delete all the nodes and relationships
graph = GraphDatabase.driver(
    uri="neo4j://localhost:7687",
    auth=("neo4j", "1234")
)

In [3]:
query = (
        "MATCH (all_nodes)"
        "OPTIONAL MATCH (all_nodes)-[all_rels]->()"
        "DELETE all_nodes, all_rels"
    )
with graph.session() as session:
    result = session.run(query)

### Load OpenNRE

In [4]:
# model = opennre.get_model('wiki80_cnn_softmax')
# model = opennre.get_model('wiki80_bert_softmax')
model = opennre.get_model('wiki80_bertentity_softmax')
# model = opennre.get_model('tacred_bert_softmax')
# model = opennre.get_model('tacred_bertentity_softmax')

2021-03-07 17:03:21,980 - root - INFO - Loading BERT pre-trained checkpoint.


# 3. Create Knowledge Graph

## 3.1 Build KG by extracting name-entities (vertices/nodes) & relations (edges/links/predicates) from textual corpus.

In [6]:
sentences = []

with open("/home/syalabi/corpus/Family.txt", "r") as f:
    sentence = f.readline()
    while sentence:
        sentences.append(sentence)
        sentence = f.readline()

for _ in sentences:
    print(_)

Happy Village is located in Florida, USA.

Mr. Wilson is a well-known local silversmith in Happy Village.

He is very very old.

When Mr. Wilson was young, he came to Happy Village to seek refuge from a war.

Mr. Wilson found his wife Mary here and raised three children.

James is the elder son of Wilson.

Tom is the younger son of Wilson.

Mr. Wilson also has a daughter named Madonna.

William is James's son.


### SpaCy: Named Entity Recognition:

https://spacy.io/api/annotation#named-entities


### OpenNRE: Relation Extraction:

https://github.com/thunlp/OpenNRE#what-is-relation-extraction


In [7]:
nlp = spacy.load('en_core_web_lg') # !python -m spacy download en_core_web_lg

exist_ent = {}
exist_relationship = []

for sentence in sentences:
    doc = nlp(sentence)
    entities = []
    names_of_entities = []
    for ent in doc.ents:
        if ent.label_ in ['PERSON', 'GPE', 'ORG'] and ent.text not in names_of_entities:
#        if ent not in entities:
            names_of_entities.append(ent.text)
            entities.append(ent)
            
    for ent in entities:
        if exist_ent.get(ent.text) is None:
            exist_ent[ent.text] = ent.text
            query = (
                "MERGE (node: "+ent.label_+" {name: $name})"
                "RETURN node"
            )
            with graph.session() as session:
                result = session.run(query, name=ent.text)
                
            print("create new node with label as {0} and name as {1}".format(ent.label_, ent.text))

    for i in range(len(entities)):
        for j in range(i + 1, len(entities)):
            text_i = entities[i].text
            text_j = entities[j].text
            loc_h = sentence.find(text_i)
            loc_t = sentence.find(text_j)
            
            result = model.infer({'text': sentence, 'h': {'pos': (loc_h, loc_h + len(text_i))},
                               't': {'pos': (loc_t, loc_t + len(text_j))}})
            
            (rel, confidence) = result[0].replace(' ', '_'), result[1]

            record = (text_i, text_j, rel, confidence)

            result_rev = model.infer({'text': sentence, 'h': {'pos': (loc_t, loc_t + len(text_j))},
                               't': {'pos': (loc_h, loc_h + len(text_i))}})
            
            (rel_rev, confidence) = result_rev[0].replace(' ', '_'), result_rev[1]

            record_rev = (text_j, text_i, rel_rev, confidence)
            
            if record not in exist_relationship:
                exist_relationship.append(record)
                if record[3] > 0.8:
                    query = (
                        "MATCH (n1 {name: $name1})"
                        "MATCH (n2 {name: $name2})"
                        "MERGE (n1) - [r:"+record[2]+"] -> (n2)"
                        "RETURN n1, n2, r"
                    )
                    with graph.session() as session:
                        result = session.run(query, name1=exist_ent[text_i], name2=exist_ent[text_j])
                        
                    print("create new relationship {0} - {1} -> {2} with confidence of {3}".format(record[0], record[2], record[1], record[3]))

            if record_rev not in exist_relationship:
                exist_relationship.append(record_rev)
                if record_rev[3] > 0.9:
                    query = (
                        "MATCH (n1 {name: $name1})"
                        "MATCH (n2 {name: $name2})"
                        "MERGE (n1) - [r:"+record_rev[2]+"] -> (n2)"   
                        "RETURN n1, n2, r"
                    )

                    with graph.session() as session:
                        result = session.run(query, name1=exist_ent[text_j], name2=exist_ent[text_i])
                        
                    print("create new relationship {0} - {1} -> {2} with confidence of {3}".format(record_rev[0], record_rev[2], record_rev[1], record_rev[3]))

create new node with label as GPE and name as Florida
create new node with label as GPE and name as USA
create new relationship Florida - country -> USA with confidence of 0.9737794995307922
create new relationship USA - contains_administrative_territorial_entity -> Florida with confidence of 0.9616801738739014
create new node with label as PERSON and name as Wilson
create new node with label as PERSON and name as Mary
create new relationship Wilson - spouse -> Mary with confidence of 0.9840330481529236
create new relationship Mary - spouse -> Wilson with confidence of 0.987860381603241
create new node with label as PERSON and name as James
create new relationship James - father -> Wilson with confidence of 0.9863383173942566
create new relationship Wilson - child -> James with confidence of 0.9927425384521484
create new node with label as PERSON and name as Tom
create new relationship Tom - father -> Wilson with confidence of 0.9853073358535767
create new relationship Wilson - child -

### Visualize created knowledge graph at: http://localhost:7474/browser/

* Username: neo4j

* Password: 1234

## 3.2 Extend/Expand KG by inferencing (auto-reasoning) new relations/entites using universal rules (static knowledge / common sense).

Automatic Reasoner

### Practise graph modification:

In [8]:
graph = GraphDatabase.driver(
    uri="neo4j://localhost:7687",
    auth=("neo4j", "1234")
)

In [9]:
# Create Happy Village node
query = (
        "CREATE (n1:GPE {name: 'Happy Village'})"
        "RETURN n1"
)

# Execuate above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

In [10]:
## Create relationship between Happy Village and Florida
query = (
        "MATCH (n1:GPE {name:'Happy Village'}), (n2:GPE {name:'Florida'})"
        "CREATE (n1)-[r:village]->(n2)"
        "RETURN n1, n2, r"
)

# Execuate above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

In [11]:
## Create relationship between Wilson and Happy Village
query = (
        "MATCH (n1:PERSON {name:'Wilson'}), (n2:GPE {name:'Happy Village'})"
        "CREATE (n1)-[r:lives]->(n2)"
        "RETURN n1, n2, r"
)

# Execuate above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

In [12]:
# Modify KG, e.g. Delete all link(s) from 'Happy Village' --to--> 'Wilson'
query = (
        "MATCH (n1:PERSON {name:'Wilson'}), (n2:GPE {name:'Happy Village'})"
        "MATCH (n1)-[r:lives]->(n2)"
        "DELETE r"
)

# Execuate above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

Your exercise: Modify KG, e.g. Delete all link(s) from 'Florida' --to--> 'Happy Village'

In [13]:
# Your exercise: Modify KG, e.g. Delete all link(s) from 'Florida' --to--> 'Happy Village'
query = (
        "MATCH (n1:GPE {name:'Happy Village'}), (n2:GPE {name:'Florida'})"
        "MATCH (n1)-[r:village]->(n2)"
        "DELETE r"
)

# Execuate above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

### Add rule set:

In [14]:
# Rule 1: Siblings relationship/link

# WHEN:
# (n1:PERSON)-[r:father]->(n2:PERSON) # n1 has a father : n2
# (n2:PERSON)-[r2:child]->(n3:PERSON) # n2 has a child  : n3
# THEN:
# n1 and n3 are siblings (bi-direcitonal)

query = (
        "MATCH (n1:PERSON)-[r:father]->(n2:PERSON)-[r2:child]->(n3:PERSON)"
        "WHERE (n1) <> (n3)"
        "MERGE (n1)-[r3:sibling]->(n3)"
        "MERGE (n3)-[r4:sibling]->(n1)"
        "RETURN n1, n3"
)

In [15]:
# Execute above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

In [16]:
# Rule 2: Mother relationship/link

# WHEN:
# (n1:PERSON)-[r:father]->(n2:PERSON)  # n1 has a father : n2
# (n2:PERSON)-[r2:spouse]->(n3:PERSON) # n2 has a spouse : n3
# THEN:
# (n1)-[r3:mother]->(n3) # n1 has a mother : n3
# (n3)-[r4:child]->(n1)  # n3 has a child  : n1

query = (
        "MATCH (n1:PERSON)-[r:father]->(n2:PERSON)-[r2:spouse]->(n3:PERSON)"
        "MERGE (n1)-[r3:mother]->(n3)"
        "MERGE (n3)-[r4:child]->(n1)"
        "RETURN n1, n3"
)

In [17]:
# Execuate above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

In [18]:
# Rule 3: Grandfather relationship/link

# WHEN:
# (n1:PERSON)-[r:father]->(n2:PERSON)  # n1 has a father : n2
# (n2:PERSON)-[r2:father]->(n3:PERSON) # n2 has a father : n3
# THEN:
# (n1)-[r3:grandfather]->(n3) # n1 has a grandfather : n3
# (n3)-[r4:grandchild]->(n1)  # n3 has a grandchild  : n1

query = (
        "MATCH (n1:PERSON)-[r:father]->(n2:PERSON)-[r2:father]->(n3:PERSON)"
        "MERGE (n1)-[r3:grandfather]->(n3)"
        "MERGE (n3)-[r4:grandchild]->(n1)"
        "RETURN n1, n3"
)

In [19]:
# Execute above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

## 3.3 Retrieve information by querying / reasoning over KG.

### Our query: 'Deep Thought, what's the relationship between Wilson and William?'

In [20]:
query = (
        "MATCH (n1 {name:'Wilson'})-[r]->(n2 {name:'William'})"
        "RETURN n1, r, n2"
)
with graph.session() as session:
    results = session.run(query)
    for result in results:
        print("Deep Thought: {0} is {1}'s {2}.".format(result['n2']['name'],
                                                      result['n1']['name'],
                                                      result['r'].type))

Deep Thought: William is Wilson's grandchild.


### Our query: 'Deep Thought, what's the relationship between James and Mary?'

In [21]:
query = (
        "MATCH (n1 {name:'James'})-[r]->(n2 {name:'Mary'})"
        "RETURN n1, r, n2"
)
with graph.session() as session:
    results = session.run(query)
    for result in results:
        print("Deep Thought: {0} is {1}'s {2}.".format(result['n2']['name'], 
                                                       result['n1']['name'],
                                                       result['r'].type))

Deep Thought: Mary is James's mother.


# 4. Knowledge Graph
### Build knowledge graph in << Animal Farm >> by  George Orwell;
### Then query various entities and relationships;

In [22]:
sentences = []
path = "/home/syalabi/corpus/Animal Farm by George Orwell.txt"

with open(path, "r") as f:
    sentence = f.readline()
    while sentence:
        sentences.append(sentence)
        sentence = f.readline()

for _ in sentences[0:5]:
    print(_)

Title: Animal Farm 

Author: George Orwell (pseudonym of Eric Blair) (1903-1950)

Chapter I

Mr Jones, of the Manor Farm, had locked the hen-houses for the night, but was too drunk to remember to shut the pop-holes.

With the ring of light from his lantern dancing from side to side, he lurched across the yard, kicked off his boots at the back door, drew himself a last glass of beer from the barrel in the scullery, and made his way up to bed, where Mrs Jones was already snoring.



In [23]:
## Initialize and refresh neo4j graph session
query = (
        "MATCH (all_nodes)"
        "OPTIONAL MATCH (all_nodes)-[all_rels]->()"
        "DELETE all_nodes, all_rels"
    )
with graph.session() as session:
    result = session.run(query)

In [24]:
## Load en_core_web_lg language model
nlp = spacy.load('en_core_web_lg') # !python -m spacy download en_core_web_lg

from spacy.pipeline import EntityRuler

ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label":"PERSON", "pattern":"Napoleon"}, 
            {"label":"PERSON", "pattern":"Squealer"}]

ruler.add_patterns(patterns)

## Initialize entity dict and relationship list
exist_ent = {}
exist_relationship = []

## Parse sentences from corpus 
for sentence in tqdm(sentences):
    try:
        doc = nlp(sentence)
        entities = []
        names_of_entities = []

        ## Extract entities from sentences
        for ent in doc.ents:
            if (ent.label_ in ['PERSON', 'GPE', 'ORG']) and (ent.text not in names_of_entities):
                names_of_entities.append(ent.text)
                entities.append(ent)

        ## Generate nodes onto neo4j graph
        for ent in entities:
            if exist_ent.get(ent.text) is None:
                exist_ent[ent.text] = ent.text
                query = (
                    "MERGE (node: "+ent.label_+" {name: $name})"
                    "RETURN node"
                )
                with graph.session() as session:
                    result = session.run(query, name=ent.text)

                print("Created new node with label as {0} and name as {1}".format(ent.label_, ent.text))

        ## Extract relationships between entities with confidence levels
        for i in range(len(entities)):
            for j in range(i + 1, len(entities)):
                text_i = entities[i].text
                text_j = entities[j].text
                loc_h = sentence.find(text_i)
                loc_t = sentence.find(text_j)

                result = model.infer({'text': sentence, 'h': {'pos': (loc_h, loc_h + len(text_i))},
                                   't': {'pos': (loc_t, loc_t + len(text_j))}})

                (rel, confidence) = result[0].replace(' ', '_'), result[1]

                record = (text_i, text_j, rel, confidence)

                result_rev = model.infer({'text': sentence, 'h': {'pos': (loc_t, loc_t + len(text_j))},
                                   't': {'pos': (loc_h, loc_h + len(text_i))}})

                (rel_rev, confidence) = result_rev[0].replace(' ', '_'), result_rev[1]

                record_rev = (text_j, text_i, rel_rev, confidence)

                if record not in exist_relationship:
                    exist_relationship.append(record)
                    if record[3] > 0.7:
                        query = (
                            "MATCH (n1 {name: $name1})"
                            "MATCH (n2 {name: $name2})"
                            "MERGE (n1) - [r:"+record[2]+"] -> (n2)"
                            "RETURN n1, n2, r"
                        )
                        with graph.session() as session:
                            result = session.run(query, name1=exist_ent[text_i], name2=exist_ent[text_j])

                        print("Create new relationship {0} - {1} -> {2} with confidence of {3}".format(record[0], record[2], record[1], record[3]))

                if record_rev not in exist_relationship:
                    exist_relationship.append(record_rev)
                    if record_rev[3] > 0.8:
                        query = (
                            "MATCH (n1 {name: $name1})"
                            "MATCH (n2 {name: $name2})"
                            "MERGE (n1) - [r:"+record_rev[2]+"] -> (n2)"   
                            "RETURN n1, n2, r"
                        )

                        with graph.session() as session:
                            result = session.run(query, name1=exist_ent[text_j], name2=exist_ent[text_i])

                        print("Create new relationship {0} - {1} -> {2} with confidence of {3}".format(record_rev[0], record_rev[2], record_rev[1], record_rev[3]))
       
    except Exception:
        continue

  0%|          | 0/1591 [00:00<?, ?it/s]

Created new node with label as PERSON and name as George Orwell
Created new node with label as PERSON and name as Eric Blair
Create new relationship George Orwell - said_to_be_the_same_as -> Eric Blair with confidence of 0.9892570376396179
Create new relationship Eric Blair - said_to_be_the_same_as -> George Orwell with confidence of 0.9881123900413513
Created new node with label as PERSON and name as Mr Jones
Created new node with label as ORG and name as the Manor Farm
Create new relationship Mr Jones - residence -> the Manor Farm with confidence of 0.9363416433334351
Created new node with label as PERSON and name as Mrs Jones
Created new node with label as PERSON and name as Major
Created new node with label as ORG and name as Willingdon Beauty
Created new node with label as ORG and name as Bluebell
Created new node with label as PERSON and name as Jessie
Created new node with label as PERSON and name as Pincher
Create new relationship Bluebell - sibling -> Jessie with confidence of

Created new node with label as PERSON and name as Animal Hero
Created new node with label as ORG and name as Foxwood Farm
Create new relationship Pilkington - sibling -> Mr Frederick with confidence of 0.8488243222236633
Create new relationship Mr Frederick - sibling -> Pilkington with confidence of 0.8108956813812256
Created new node with label as ORG and name as Frederick, Snowball
Create new relationship Frederick, Snowball - residence -> Foxwood with confidence of 0.9234345555305481
Create new relationship Frederick, Snowball - residence -> Pilkington with confidence of 0.8385541439056396
Created new node with label as ORG and name as Frederick of Pinchfield Farm
Create new relationship Animal Farm - owned_by -> Mr Frederick with confidence of 0.9540413618087769
Created new node with label as PERSON and name as Chapter VIII

Created new node with label as ORG and name as Crown Derby
Created new node with label as ORG and name as Ducklings
Created new node with label as ORG and name

### Add Ruleset

In [25]:
# Rule 1: Resident & Animal Farm relationship/link

query = (
        "MATCH (n1:PERSON), (n2 {name:'Animal Farm'}), (n3 {name:'Napoleon'})"
        "MERGE (n1)-[r1:resident]->(n2)"
        "MERGE (n1)-[r2:comrade]->(n3)"
        "MERGE (n3)-[r3:comrade]->(n1)"
)
# Execute above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

In [26]:
# Rule 2: Resident and participant relationship/link

query = (
        "MATCH (n1 {name:'Napoleon'})-[r1]->(n2:PERSON)-[r2:resident]->(n3 {name:'Animal Farm'})"
        "MERGE (n4 {name:'Rebellion'})-[r5:participant]->(n2)"
)

# Execute above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

In [27]:
# Rule 3: Delete sibling and spouse links relationship/link

query = (
        "MATCH (n1)-[r1:sibling]->(n2:PERSON), (n3:PERSON)-[r2:spouse]->(n4:PERSON)"
        "DELETE r1, r2"
)

# Execute above query to modify neo4j db:
with graph.session() as session:
    results = session.run(query)

---

In [28]:
query = (
        "MATCH (n1 {name:'Napoleon'})-[r]->(n2 {name:'Animal Farm'})"
        "RETURN n1, r, n2"
)
with graph.session() as session:
    results = session.run(query)
    for result in results:
        print("Deep Thought: {0} is {1} of {2}.".format(result['n1']['name'], 
                                                           result['r'].type, 
                                                           result['n2']['name']))

Deep Thought: Napoleon is field_of_work of Animal Farm.
Deep Thought: Napoleon is resident of Animal Farm.


In [29]:
query = (
        "MATCH (n1 {name:'Napoleon'})-[r]->(n2:PERSON)"
        "RETURN n1, r, n2"
)
with graph.session() as session:
    results = session.run(query)
    for result in results:
        print("Deep Thought: {0} is {1} of {2}.".format(result['n1']['name'], 
                                                           result['r'].type, 
                                                           result['n2']['name']))

Deep Thought: Napoleon is said_to_be_the_same_as of Clover.
Deep Thought: Napoleon is said_to_be_the_same_as of Mr Jones's.
Deep Thought: Napoleon is father of Mr Jones.
Deep Thought: Napoleon is residence of Foxwood.
Deep Thought: Napoleon is military_rank of Major.
Deep Thought: Napoleon is comrade of Muriel.
Deep Thought: Napoleon is comrade of Alfred Simmonds.
Deep Thought: Napoleon is comrade of Mollie.
Deep Thought: Napoleon is comrade of Animalism.
Deep Thought: Napoleon is comrade of Foxwood.
Deep Thought: Napoleon is comrade of Mrs Jones's.
Deep Thought: Napoleon is comrade of Jones.
Deep Thought: Napoleon is comrade of A.
.
Deep Thought: Napoleon is comrade of George Orwell.
Deep Thought: Napoleon is comrade of Mr Jones's.
Deep Thought: Napoleon is comrade of Chapter III
.
Deep Thought: Napoleon is comrade of Clover.
Deep Thought: Napoleon is comrade of Chapter VIII
.
Deep Thought: Napoleon is comrade of John Bull'.
Deep Thought: Napoleon is comrade of Horse Slaughterer.
Deep

In [30]:
query = (
        "MATCH (n1:PERSON)-[r:resident]->(n2 {name:'Animal Farm'})"
        "RETURN n1, r, n2"
)
with graph.session() as session:
    results = session.run(query)
    for result in results:
        print("Deep Thought: {0} is {1} of {2}.".format(result['n1']['name'], 
                                                           result['r'].type, 
                                                           result['n2']['name']))

Deep Thought: Mrs Jones's is resident of Animal Farm.
Deep Thought: Chapter III
 is resident of Animal Farm.
Deep Thought: A.
 is resident of Animal Farm.
Deep Thought: Foxwood is resident of Animal Farm.
Deep Thought: Mr Pilkington is resident of Animal Farm.
Deep Thought: a Mr Frederick is resident of Animal Farm.
Deep Thought: Frederick is resident of Animal Farm.
Deep Thought: Pilkington is resident of Animal Farm.
Deep Thought: Julius Caesar's is resident of Animal Farm.
Deep Thought: Mr Jones--'One Thousand is resident of Animal Farm.
Deep Thought: Chapter VI
 is resident of Animal Farm.
Deep Thought: Whymper is resident of Animal Farm.
Deep Thought: Mr Frederick is resident of Animal Farm.
Deep Thought: Animal Hero is resident of Animal Farm.
Deep Thought: Chapter VIII
 is resident of Animal Farm.
Deep Thought: Sleeps is resident of Animal Farm.
Deep Thought: Alfred Simmonds is resident of Animal Farm.
Deep Thought: Horse Slaughterer is resident of Animal Farm.
Deep Thought: Joh

In [31]:
query = (
        "MATCH (n1 {name:'Napoleon'})-[r]->(n2 {name:'Snowball'})"
        "RETURN n1, r, n2"
)
with graph.session() as session:
    results = session.run(query)
    for result in results:
        print("Deep Thought: {0} has {1} {2}.".format(result['n1']['name'], 
                                                           result['r'].type, 
                                                           result['n2']['name']))

Deep Thought: Napoleon has comrade Snowball.


In [32]:
query = (
        "MATCH (n1 {name:'Rebellion'})-[r]->(n2)"
        "RETURN n1, r, n2"
)
with graph.session() as session:
    results = session.run(query)
    for result in results:
        print("Deep Thought: {0} has {1} {2}.".format(result['n1']['name'], 
                                                           result['r'].type, 
                                                           result['n2']['name']))

Deep Thought: Rebellion has participant Mrs Jones's.
Deep Thought: Rebellion has participant Chapter III
.
Deep Thought: Rebellion has participant A.
.
Deep Thought: Rebellion has participant Foxwood.
Deep Thought: Rebellion has participant Mr Pilkington.
Deep Thought: Rebellion has participant a Mr Frederick.
Deep Thought: Rebellion has participant Frederick.
Deep Thought: Rebellion has participant Pilkington.
Deep Thought: Rebellion has participant Julius Caesar's.
Deep Thought: Rebellion has participant Mr Jones--'One Thousand.
Deep Thought: Rebellion has participant Chapter VI
.
Deep Thought: Rebellion has participant Whymper.
Deep Thought: Rebellion has participant Mr Frederick.
Deep Thought: Rebellion has participant Animal Hero.
Deep Thought: Rebellion has participant Chapter VIII
.
Deep Thought: Rebellion has participant Sleeps.
Deep Thought: Rebellion has participant Alfred Simmonds.
Deep Thought: Rebellion has participant Horse Slaughterer.
Deep Thought: Rebellion has partici